In [1]:
# Setup display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","Plots","PyCall","JSON","JLD2","FileIO","ImageMagick"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Base.Iterators: flatten
using Statistics: mean
using Plots;
using FileIO;
using MAT;
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, Param,progress, sgd,dropout, relu,
Data,sigm, xavier,gpu,softmax
import Knet:param,param0
using Images;
using PyCall,JSON,JLD2,Random
@pyimport numpy as np 

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:400
└ @ PyCall /home/ege/.julia/packages/PyCall/RQjD7/src/PyCall.jl:400


In [27]:
#=atype()=(gpu() >= 0 ? KnetArray{Float32} : Array{Float32})
param(d...; init=xavier, atype=atype())=Param(atype(init(d...)))
param0(d...; atype=atype())=param(d...; init=zeros, atype=atype)=#

In [2]:
struct Chain
    layers
    Chain(layers...)=new(layers)
end
(c::Chain)(x) = (for layer in c.layers; x=layer(x) end;x)#if printsize==1 println(size(x)) end;end; x) #layerların çıktılarını hesaplayıp son layerın çıktısını veriyor
(c::Chain)(x,y)=nll(c(x),y) #loss
(c::Chain)(d::Data)=mean(c(x,y) for (x,y) in d) #batch için mean loss hesapla

In [4]:
struct Fully; w;b;func;end;
Fully(m::Int,n::Int,func)=Fully(param(n,m),param0(n),func)
(ful::Fully)(x)=ful.func.(ful.w*x.+ful.b)

In [3]:
struct Conv; w;b;func;
    Conv(w,b)=new(Param(w),Param(b),relu)
    Conv(w,b,func)=new(Param(w),Param(b),func)
    end;
(c::Conv)(x)=c.func.(conv4(c.w,x,padding=1,mode=1).+c.b)
#(c::Conv)(x)=conv4(c.w,x,padding=1,mode=1).+c.b

In [4]:
struct Pool end
(p::Pool)(x)=pool(x)

In [5]:
struct Dense; w;b;func;
    Dense(w,b)=new(Param(w),Param(b),relu)
    Dense(w,b,func)=new(Param(w),Param(b),func)
    Dense(x::Int,y::Int,func)=new(param(y,x),param0(y),func)
    end;
function (den::Dense)(x) 
    matrix_multip=den.w*mat(x).+den.b
    if den.func==relu
        return den.func.(matrix_multip)
    else
        return den.func(matrix_multip)
    end
end

In [6]:
weight_dict=matread("imagenet-vgg-verydeep-16.mat")
layers=weight_dict["layers"]
w_arr=[]
for layer in layers
    if occursin("conv",layer["name"])
        w=layer["weights"]
        push!(w_arr,w[1])
        b=reshape(w[2], (1,1,length(w[2]),1))
        push!(w_arr,b)
        elseif occursin("fc",layer["name"])
            w=layer["weights"]
            w[1] = transpose(mat(w[1]))
            push!(w_arr,w[1])
            push!(w_arr,w[2])
        end
end

In [7]:
function sof(x)
    exp.(Knet.logp(x,dims=1))
end

sof (generic function with 1 method)

In [15]:
Vgg_model=Chain(Conv(w_arr[1],w_arr[2]),Conv(w_arr[3],w_arr[4]),Pool(),   #,  #layer1
    Conv(w_arr[5],w_arr[6]),Conv(w_arr[7],w_arr[8]),Pool(),              #layer2
    Conv(w_arr[9],w_arr[10]),Conv(w_arr[11],w_arr[12]),Conv(w_arr[13],w_arr[14]),Pool(),       #layer3
    Conv(w_arr[15],w_arr[16]),Conv(w_arr[17],w_arr[18]),Conv(w_arr[19],w_arr[20]),Pool(),       #layer4
    Conv(w_arr[21],w_arr[22]),Conv(w_arr[23],w_arr[24]),Conv(w_arr[25],w_arr[26]),Pool(),Dense(8192,3,softmax))       #layer5
 #   Dense(w_arr[27],w_arr[28]),Dense(w_arr[29],w_arr[30]),Dense(w_arr[31],w_arr[32],softmax));#prediction burada

In [47]:
function preprocesscroppedimage(img::AbstractMatrix{<:AbstractRGB})
    z = (channelview(img).* 255)# .- 128)./128;
    _atype(permutedims(z, (2, 3, 1))[:,:,:,:])
    z=reverse(z,dims=3)
    #_atype(z[:,:,:,:])
end

preprocesscroppedimage (generic function with 1 method)

In [8]:
function image_data(img,image_size)
    global _imgcache
    if !@isdefined(_imgcache); _imgcache = Dict(); end
        a0=img
        new_size = ntuple(i->div(size(a0,i)*image_size,minimum(size(a0))),2)
        a1 = Images.imresize(a0, new_size)
        i1 = div(size(a1,1)-image_size,2)
        j1 = div(size(a1,2)-image_size,2)
        b1 = a1[i1+1:i1+image_size,j1+1:j1+image_size]
        # ad-hoc solution for Mac-OS image
        macfix = convert(Array{FixedPointNumbers.Normed{UInt8,8},3}, channelview(b1))
        c1 = permutedims(macfix, (3,2,1))
        d1 = convert(Array{Float32}, c1)
        e1 = reshape(d1[:,:,1:3], (image_size,image_size,3,1))
        averageImage=[123.68 116.779 103.939]
        f1 = (255 * e1)#[:,:,1,:] .- averageImage)
        f1[:,:,1,:].-=averageImage[1]
        f1[:,:,2,:].-=averageImage[2]
        f1[:,:,3,:].-=averageImage[3]
        g1 = permutedims(f1, [2,1,3,4])
    return g1
end

image_data (generic function with 1 method)

In [18]:
img=image_data(load("dalmat.jpg"),128);

In [19]:
pred=Vgg_model(img)

3×1 Array{Float32,2}:
 1.397924e-5 
 0.99997556  
 1.0462122e-5

In [22]:
confidence,class_index=findmax(pred)
class_index=class_index[1]
println(confidence)
f = open("imagenet_classes.txt")
lines=readlines(f);
count=1
for i in lines
    if count==class_index
        println(i)
        break
    end
    count+=1
end
close(f)

0.9999521
zebra


In [10]:
img2=image_data(load("dalmat.jpg"),128);
img3=image_data(load("bee.jpg"),128);
img_d=cat(img,img2,dims=4);
img_d=cat(img_d,img3,dims=4);
img_d=cat(img_d,img_d,dims=4);
summary(img_d)

"128×128×3×6 Array{Float32,4}"

In [11]:
batch_train=Knet.minibatch(img_d,[1,1,1,1,1,1],2,xtype=Array{Float32},ytype=Array{Int32});

In [12]:
# For running experiments
function trainresults(model,dtrn,ep,optimizer,learning_r)
    train_iter(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
                
        r = ((model(dtrn),zeorone(model,dtrn))
             for x in train_iter(length(dtrn), progress(optimizer(model,repeat(dtrn,ep),lr=learning_r))))         
    return r
end

trainresults (generic function with 1 method)

In [79]:
# For running experiments
function trainresults2(model,dtrn,ep,optimizer,learning_r)
    train_iter(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
                
        r = ((model(dtrn), zeroone(model,dtrn))
             for x in train_iter(length(dtrn), progress(optimizer(model,repeat(dtrn,ep),lr=learning_r))))
        r = reshape(collect(Float32,flatten(r)),(2,:))          
    return r
end

trainresults2 (generic function with 1 method)

In [57]:
dense_model=Chain(Dense(2,10,relu),Dense(10,20,softmax));

In [53]:
x_data=convert(Array{Float32},rand(2))
for i in 1:1999
    x_data=cat(x_data,convert(Array{Float32},rand(2)),dims=2)
end
y_data=ones(2000)

batch_dense=Knet.minibatch(x_data,y_data,10,ytype=Array{Int32});

In [48]:
a=rand(2)
cat(a,a,dims=2)

2×2 Array{Float64,2}:
 0.294781  0.294781
 0.113139  0.113139

In [74]:
results=trainresults2(dense_model,batch_dense,20,sgd,0.1);

2.90e+00  100.00%┣█████████████████▉┫ 4000/4000 [00:01/00:01, 3297.55i/s]


In [77]:
findmax(dense_model(rand(2,1)))

(0.9984130437317741, CartesianIndex(1, 1))

In [80]:
r=trainresults2(Vgg_model,batch_train,10,sgd,0.1);

1.33e+00  100.00%┣████████████████████████┫ 30/30 [08:50/08:50, 17.67s/i]


In [87]:
Vgg_model(img)

3×1 Array{Float32,2}:
 0.9331823    
 2.4477837e-20
 0.06681774   